###### Trabajo relaizado en google colab , aprovechando su rapidez a la hora de ejecutar los codigos  de el webscrapeo

In [ ]:
# !pip install google.colab 

In [ ]:
from google.colab import drive  #importamos esta libreria que conecta el drive con el colab

El siguiente codigo es una especie  de permiso de acceder a nuestro drive para que haga uso de nuestros archivos
pasos para acceder al drive  
1-AL correr el codigo aparecera una ventana donde espera que coloquemos un permiso para continuar 
2 darle click al enlace que aparece 
3-selecionar la direcion de correo electronico que se usara para acceder al drive 
4-automaticamente nos saldra un enlace copiar y pegar en la ventana que aparece cuando corrimos el codigo


In [ ]:
drive.mount('/content/drive')  

In [ ]:
from bs4 import BeautifulSoup #importamos la libreria que nos ayudara con la extraccion de datos de la pagina unitprotein
import requests 
import pandas as pd #para el manejo de data frame
import numpy as np 
from tqdm.notebook import tqdm_notebook as tqdm #importamos esta libreria nueva para medir,validar el rrecorrido de la extraccion 

In [ ]:
ruta='/content/drive/MyDrive/webs/Genes identificados con vecinos.xlsx' #leo el archivo donde estan todos los genees

In [ ]:
#lee todas las hojas del excel que colocamos en un data frame por separado
#sheet name  = 'va el nombre de la hoja a leer'
data1=pd.read_excel(ruta,sheet_name='C1',skiprows=4)
data2=pd.read_excel(ruta,sheet_name='C2',skiprows=1)
data3=pd.read_excel(ruta,sheet_name='C3',skiprows=1)
data4=pd.read_excel(ruta,sheet_name='C4-M',skiprows=1)
data5=pd.read_excel(ruta,sheet_name='C5',skiprows=1)
data6=pd.read_excel(ruta,sheet_name='C6',skiprows=1)
data7=pd.read_excel(ruta,sheet_name='C7-J',skiprows=1)
data8=pd.read_excel(ruta,sheet_name='C8-N',skiprows=1)
data9=pd.read_excel(ruta,sheet_name='C9',skiprows=1)
data10=pd.read_excel(ruta,sheet_name='C10',skiprows=1)
data11=pd.read_excel(ruta,sheet_name='C11',skiprows=1)

In [ ]:
#pd.concat(['']),....
#con ese codigo lo que hace es que de cada data frame que hemos creado extraeme solo la columna 'Gen abre'
#y asi va concatenando todos los datas frames por fila que contienes cada una de las hojas del excel
#reset_index(drop=True) ordena bien los indices
data_gen=pd.concat([data1['Gen Abrev'],data2['Gen Abrev'],data3['Gen Abrev'],data4['Gen Abrev'],data5['Gen Abrev'],
          data6['Gen Abrev'],data7['Gen Abrev'],data8['Gen Abrev'],data9['Gen Abrev'],data10['Gen Abrev'],
          data11['Gen Abrev']],axis=0).reset_index(drop=True)

In [ ]:
data_genes=pd.DataFrame({'genes':data_gen}) #lo que concatene lo paso a un data frame

In [ ]:
#data_genes.loc[data_genes.genes.duplicated(),:]#lo que hace es  me da la cantidad de valores unicos 

In [ ]:
#usaremos como path 
path='https://www.uniprot.org/uniprot/?query=&sort=score'

In [ ]:

corte_link=path.split('&')#split corta cadenas
corte_link

LIMPIEZA DE LOS NOMBRES ABREVIADOS
hay nombres abreviados con parentesis,los eliminamos y nos quedamos con el principal por ejemplo:

C3H1orf43(c1orf43) ==> C3H1orf43

In [ ]:
#lo que hace esque va buscar los que cumplas con la expresion regular requerida los va eliminar y solo 
#se quedaracon el primer nombre que esta sin parentesis
import re
data_genes1=data_genes.applymap(lambda x:re.sub('\(.*?\)','',x).strip())
data_genes1.head()

In [ ]:
data_genes1.shape #verificamos cuantos registros tenemos

In [ ]:
data_genes1.genes.nunique() #cuanta los genes unicos , que hay en la data  

In [ ]:
espacio=data_genes1['genes'].apply(lambda x:x.__contains__(' '))
sum(espacio) #CONTARA  DE LOS GENES QUE EN SU NOMBRE HAY ESPACIOS 

In [ ]:
data_genes2=data_genes1.applymap(lambda x:x.strip().replace(' ',''))

In [ ]:
data_genes2.genes.head() #MUESTRA LO 5 PRIMEROS  

**Concatenamos el link con la abreviatura del gen**

In [ ]:
genes_path=data_genes2.applymap(lambda x:corte_link[0]+x+'+bos+taurus&'+corte_link[1])#concatenamos el link segun el patron para hacer el scraping
genes_path.head()

## **CREAMOS LA PRIMERA FUNCION**

In [ ]:
import gc #importamos la libreria gc para liberar memoria

In [ ]:
def link_page1(data):
  lista2=[] #creamos una lista vacia
  lista_dat=data.values.tolist() #lo transformamos la data a una lista para hacerlo mas facil con el tolist
  for i in tqdm(range(0,len(lista_dat)),total=len(lista_dat)):
    try:
      p12=requests.get(lista_dat[i]) #para jalar el link para correr cada gen
      if p12.status_code==200:
        s=BeautifulSoup(p12.text,'lxml') 
        secciones=s.find('div',attrs={'class':"main-aside"}) #empezamos el webscrapeo 
        secc1=secciones.find('td',attrs={'class':'entryID'}) #escogemos el primer cuadro con el "td"
    except Exception:
      entry,entry_name=None,None # si hay un error de la funcion saldra None 

 #  s=BeautifulSoup(p12.text,'lxml') 
  #  secciones=s.find('div',attrs={'class':"main-aside"})
   # secc1=secciones.find('td',attrs={'class':'entryID'})
    try :
 #       hrf='https://www.uniprot.org/'+secc1.a.get('href')
        entry=secc1.get_text()                           # con el get_text() jalamos el primer nombre
        entry_name=secciones.find('tbody').find_all('td')[2].get_text() # jalamos todo lo que esta dentro del body 
    except :
        entry,entry_name=None,None # si hay un error de la funcion saldra None
    lista2.append([entry,entry_name])
    gc.collect()
  return lista2

In [ ]:
salida=link_page1(genes_path.iloc[84:100,0]) #probamos algunas partes pequeñas para ver si funciona

In [ ]:
salida1=link_page1(genes_path.iloc[90:100,0]) #probamos algunas partes pequeñas para ver si funciona

## Pobramos haciendo la funcion por partes 
Lo probamos por partes para que la funciona funcione de manera eficiente y mi computadora no se reinicie 

In [ ]:
salida_total1=link_page1(genes_path.iloc[0:506,0]) #desde la linea 0 hasta la 506

In [ ]:
salida_total2=link_page1(genes_path.iloc[506:1012,0]) #desde la linea 506 hasta la 1012

In [ ]:
salida_total3=link_page1(genes_path.iloc[1012:1518,0]) #desde la linea 1012 hasta la 1518

In [ ]:
salida_total4=link_page1(genes_path.iloc[1518:2024,0]) #desde la linea 1518 hasta la 2024

In [ ]:
salida_total5=link_page1(genes_path.iloc[2024:,0])

In [ ]:
for i in [salida_total2,salida_total3,salida_total4,salida_total5]:
  salida_full.extend(i) #unimos todo en una sola lista

In [ ]:
len(salida_full) # vemos si esta toda la data

In [ ]:
data_entry=pd.DataFrame(salida_full,columns=['Entry','Entry Name']) # armamos nuestro data frame

In [ ]:
data_entry.head(10) # los primeros 10 entry y entry name

In [ ]:
data_entry.isnull().sum()#Comprorbamos cuantos valores nulos hay en ddata_entry

In [ ]:
data_entry['link_post']=data_entry['Entry'].apply(lambda x:'https://www.uniprot.org/uniprot/'+x if x!=None else None) #Se concatena el nombre de entry en a la variable link_post 

In [ ]:
data_entry.head()#Revisamos los encabezados de entry

In [ ]:
data_concat_genes=pd.concat([data_genes2['genes'],data_entry],axis=1)#Conactenamos las cadenas de los names de data_genes2 e la etiqueta genes con data entry

In [ ]:
data_concat_genes.head()# vemos los encabezados resultado de las cadenas concatenadas

In [ ]:
data_entry.to_csv('data_entry.csv',header=True,index=None)#Se abre un csv con el nombre data_entry

In [ ]:
data_concat_genes.to_csv('data_concat_genes.csv',header=True,index=None)#Se abre un CSV para alamaccnar los datos que nos piden de genes